# 1. SETTINGS

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler
import scipy.stats
import os
import time

In [ ]:
# helper functions
import functions
from functions import *

In [ ]:
# pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# white axis labeles
params = {'ytick.color' :     'w',
          'xtick.color' :     'w',
          'axes.labelcolor' : 'w',
          'axes.edgecolor' :  'w'}
plt.rcParams.update(params)

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# garbage collection
import gc
gc.enable()

# 2. DATA PARTITIONING

In [ ]:
# import CSV
df = pd.read_csv('../data/data_v3.csv')
print(df.shape)

In [ ]:
# target variable
target = 'fraud'

In [ ]:
# partitioning
train = df[df[target].isnull() == False]
test  = df[df[target].isnull() == True]
print(train.shape)
print(test.shape)

In [ ]:
# target variable
y = train[target]
del train[target], test[target]

# 3. MODELING

In [ ]:
# drop bad features
excluded_feats = ['id']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

In [ ]:
### PARAMETERS

# settings
cores = 12
seed  = 23

# cross-validation
num_folds = 5
shuffle   = True

# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'subsample':         0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.05,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
}

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)

In [ ]:
# placeholders
clfs = []
valid_profit = np.zeros(num_folds) 
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [ ]:
# SMOTE settings
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 23, n_jobs = 10, sampling_strategy = 0.1)

In [ ]:
########### FUNCTION FOR TEST TIME AUGMENTATION

# creates multiple versionf of test data (with noise)
# averages predictions over the created samples

def predict_proba_with_tta(X_test, model, num_iteration, alpha = 0.01, n = 4, seed = 0):
    
    # set random seed
    np.random.seed(seed = seed)
    
    # original prediction
    preds = model.predict_proba(X_test, num_iteration = num_iteration)[:, 1] / (n + 1)
     
    # select numeric features
    num_vars = [var for var in X_test.columns if X_test[var].dtype != "object"]
    
    # synthetic predictions
    for i in range(n):
        
        # copy data
        X_new = X_test.copy()
                  
        # introduce noise
        for var in num_vars:
            X_new[var] = X_new[var] + alpha * np.random.normal(0, 1, size = len(X_new)) * X_new[var].std()
            
        # predict probss
        preds_new = model.predict_proba(X_new, num_iteration = num_iteration)[:, 1]
        preds += preds_new / (n + 1)
    
    # return probs
    return preds

In [ ]:
### CROSS-VALIDATION LOOP
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    
    # scale data
    scaler   = RobustScaler()
    trn_x    = scaler.fit_transform(trn_x)
    val_x    = pd.DataFrame(scaler.transform(val_x),          columns = features)
    tmp_test = pd.DataFrame(scaler.transform(test[features]), columns = features)

    # augment training data with SMOTE
    trn_x, trn_y = sm.fit_sample(trn_x, trn_y)
    trn_x = pd.DataFrame(trn_x, columns = features)
    trn_y = pd.Series(trn_y)
      
    # add noise to train to reduce overfitting
    #trn_x += np.random.normal(0, 0.01, trn_x.shape)
    
    # mean target encoding
    #trn_x, val_x, tmp_test = mean_target_encoding(trn_x, val_x, test, features = ['trustLevel'], target = 'fraud', folds = 5)
    #features = [f for f in trn_x.columns if f not in excluded_feats]
        
    # train lightGBM
    print('Custom early stopping: select the best out of %.0f iterations...' % max_rounds)
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = prediction_reward, 
                  #eval_metric           = "logloss", 
                  #early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # find the best iteration
    #best_iter = clf.best_iteration_
    best_iter = np.argmax(clf.evals_result_['valid_1']['profit']) + 1
    print('Best iteration is:')
    print('[' + str(best_iter) + ']   valid_1 profit: ' + 
          str(prediction_reward(val_y, clf.predict_proba(val_x, num_iteration = best_iter)[:, 1])[1].astype('int')))
       
    # predictions
    #preds_oof[val_idx]    = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    preds_oof[val_idx]    = predict_proba_with_tta(val_x, model = clf, num_iteration = best_iter, 
                                                   alpha = 0.001, n = 10, seed = 7)
    valid_profit[n_fold]  = prediction_reward(val_y, preds_oof[val_idx])[1]
    #preds_test           += clf.predict_proba(tmp_test, num_iteration = best_iter)[:, 1] / folds.n_splits 
    preds_test           += predict_proba_with_tta(tmp_test, model = clf, num_iteration = best_iter, 
                                                   alpha = 0.001, n = 10, seed = 7) / folds.n_splits 

    ## importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature'] = features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold'] = n_fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('----------------------')
    print('FOLD%2d: PROFIT = %.0f' % (n_fold + 1, valid_profit[n_fold]))
    print('----------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    # uncomment with mean target encoding
    #features = [f for f in train.columns if f not in excluded_feats]
    
    
# print overall performance    
cv_perf = np.sum(valid_profit)
print('----------------------')
print('TOTAL PROFIT = %.0f' % cv_perf)
print('----------------------')

In [ ]:
##### RECHECK PROFIT  
prediction_reward(y, preds_oof)


###### TRACKING RESULTS

# V1: lgb, 5 folds, default features:   80
# V2: add feature:  no. total items:   250
# V3: use logloss for ES, not profit:  260
# V4: add feature: no. weird actions:  275
# V5: custom earlystop for profit:     320
# V6: add SMOTE for minority class:    335
# V7: add robust data scaling:         350 

In [ ]:
##### VARIABLE IMPORTANCE

# load importance    
top_feats = 100
cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False)[0:top_feats].index
importance = importances.loc[importances.Feature.isin(cols)]
    
# plot variable importance
plt.figure(figsize = (10,6))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False))
plt.tight_layout()

# save plot as pdf
plt.savefig('../var_importance.pdf')

In [ ]:
##### OPTIMIZE CUTOFF

# set step
step = 100

# search
cutoffs = []
profits = []
for i in range(0, step):
    cutoffs.append(i / step)
    profits.append(recompute_reward(y, preds_oof, cutoff = cutoffs[i]))
        
# results
plt.figure(figsize = (10,4))
sns.lineplot(x = cutoffs[10:step], y = profits[10:step], color = 'red')
plt.tight_layout()
plt.axvline(x = cutoffs[np.argmax(profits)], color = 'black', linestyle = '--')
print('- optimal cutoff = %.4f' % cutoffs[np.argmax(profits)])
print('- optimal profit = %.4f' % profits[np.argmax(profits)])
plt.savefig('../cutoff_selection.pdf')

# 4. SUBMISSION

In [ ]:
# file name
model = 'lgb_v7'
perf  = str(round(cv_perf, 0).astype('int'))
name  = model + '_' + perf

In [ ]:
# export OOF preds
oof = pd.DataFrame({'id': train['id'], 'fraud': preds_oof})
oof.to_csv('../oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [ ]:
# check submission
sub = pd.DataFrame({'id': test['id'], 'fraud': preds_test})
sub['fraud'] = np.round(sub['fraud']).astype('int')
sub.head()

In [ ]:
# export submission
sub = sub[['fraud']]
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.shape

In [ ]:
# check correlation with previous submission
prev_sub = pd.read_csv('../submissions/lgb_v6_335.csv')
cor = np.sum(prev_sub[target] == sub.reset_index()[target]) / len(sub)
print("Share of the same predictions: " + str(np.round(cor, 6)))